# After study and understand the data with Panda, I did smililar data engineering here with Pyspark 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType, StringType, StructType, StructField
from pyspark.ml.feature import StringIndexer, VectorAssembler, QuantileDiscretizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark import SparkContext
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
# build spark session
spark = SparkSession.builder.appName("Spark on santander-customer-satisfaction").getOrCreate()

In [ ]:
# Loading data to spark session
train_spark = spark.read.csv("../input/santander-customer-satisfaction/train.csv", header="true", inferSchema="true")
test_spark = spark.read.csv("../input/santander-customer-satisfaction/test.csv", header="true", inferSchema="true")

# Loading data panda to simplify data visuallization 


train = pd.read_csv('../input/santander-customer-satisfaction/train.csv')

test = pd.read_csv('../input/santander-customer-satisfaction/test.csv')

 #  1.Overview and understand data 

In [ ]:
train_spark.printSchema()

#summary of the data

# Transfer spark to panda dataframe due to bad visibility in the kaggle Kernel. Maybe there is solution to have better display. 

In [ ]:
train_spark.describe().limit(5).toPandas()

In [ ]:
train_spark.describe().toPandas()

In [ ]:
train_spark.limit(5).toPandas()

> Here I check number of rows for each ID?
* Conclusion: there is one row for one ID



In [ ]:
from pyspark.sql.functions import col
df_ID_count = train_spark.groupBy("ID").count().orderBy('count', ascending=False)
#df_ID_count = train_spark.groupBy("ID").count().filter("`count` >= 10").sort(col("count").desc())
df_ID_count.show(20)

> function below will go through each column one by one to do the describe, it is really good pratic but it takes long time to process. Should not be used unless it requires to go through each parameters one by one. 

In [ ]:
##########warning take long time do not start it unless need to review each column on by one #####################
def describe_Column(df, column, target='TARGET', numRows=20):
    df.groupby(column).agg(F.count(column).alias('count'), 
                           F.mean(target).alias('mean'), 
                           F.stddev(target).alias('stddev'),
                           F.min(target).alias('min'), 
                           F.max(target).alias('max')
                          ).orderBy('count', ascending=False).show(numRows)
    

#for column, typ in train_spark_reduce.dtypes:
    #print(column)
    #describe_Column(train_spark_reduce, column)
    # please comment out lines above to use 

In [ ]:
#check the type
type(df_ID_count)

In [ ]:
df_Target_count = train_spark.groupBy("TARGET").count()
df_Target_count.show()
type(df_Target_count)

Understand the sample ratio. Balance between unsatisfied customers (1) and satisfied customers (0).

In [ ]:
import pyspark.sql.functions as f
from pyspark.sql.window import Window
df_Target_count = df_Target_count.withColumn('ratio', f.col('count')/f.sum('count').over(Window.partitionBy()))
df_Target_count.orderBy('ratio', ascending=False).show()

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

print('Histogram plot ')
sns.countplot('TARGET', data=train_spark.toPandas())
plt.title('Target size', fontsize=14)
plt.show()

print("Dataset is imbalanced")

# 2.Processing and cleaning data

# Data cleanning (remove irrelevant)

In [ ]:
#Assuming ID is not correlated with customer satisfaction so i drop it
train_spark_NoID = train_spark.drop('ID')
train_spark_NoID.limit(5).toPandas()

# Data cleanning (drop duplicate rows)
* maybe it is not good to drop, since different customer may have exist same profile

In [ ]:
print("Before dorp duplicate count: ",train_spark_NoID.count())
#drop duplicate 
train_spark_NoID_NoDupRow = train_spark_NoID.dropDuplicates()
print("After dorp duplicate count: ",train_spark_NoID_NoDupRow.count())

In [ ]:
train_spark_NoID_NoDupRow.distinct().count()

# Data cleanning (drop duplicate columns)

In [ ]:
# remove duplicated columns
remove = []
cols = train.columns
for i in range(len(cols)-1):
    v = train[cols[i]].values
    for j in range(i+1,len(cols)):
        if np.array_equal(v,train[cols[j]].values):
            remove.append(cols[j])

In [ ]:
print("Before dorp duplicate column count: ",len(train_spark_NoID_NoDupRow.columns))
train_spark_NoID_NoDup = train_spark_NoID_NoDupRow.drop(*remove)
print("After dorp duplicate column count: ",len(train_spark_NoID_NoDup.columns))

# Data cleanning (remove distinct count =1 )
* code below used pure spark dataframe, if it is in panda dataframe, it will be calcuate each column STD.
* Scripte is really slow, need improvement
* script below only get one distinct column therefore, I remove it from running 

print("Before dorp column count: ",len(train_spark_NoID_NoDup.columns))
#from pyspark.sql.functions import * is need for the countDistinct
from pyspark.sql.functions import *
#apply countDistinct on each column
col_counts = train_spark_NoID_NoDup.agg(*(countDistinct(col(c)).alias(c) for c in train_spark_NoID_NoDup.columns)).collect()[0].asDict()

#select the cols with Distinct count=1 in an array
cols_to_drop = [col for col in train_spark_NoID_NoDup.columns if col_counts[col] == 1 ]

#drop the selected column
train_spark_drop1Distinct = train_spark_NoID_NoDup.drop(*cols_to_drop)
print('Number of cols dropped: ',len(cols_to_drop))
print("After dorp column count after removing distince count =1 : ",len(train_spark_drop1Distinct.columns))

In [ ]:
train_spark_drop1Distinct = train_spark_NoID_NoDup

# Data cleanning (replace strange value in columns)
* the value -99999 looks werid/strange value, may need to replace

In [ ]:
# count name of werid number for each columns and sort by count
count_series = train[train<-100000].count()
df_count=count_series.to_frame().T
df_count.max().sort_values(ascending=False).head()



In [ ]:
# only var3 column has the strange value -999999
# train_spark_drop1Distinct.filter(train_spark_drop1Distinct.var3 == -999999).toPandas()

In [ ]:
train_spark_drop = train_spark_drop1Distinct.withColumn('var3', F.when(train_spark_drop1Distinct['var3']< -100000,train['var3'].median()).otherwise(train_spark_drop1Distinct['var3']))

In [ ]:
#train_spark_drop.filter(train_spark_drop.var3 == -999999).toPandas()
train_spark_drop.describe().toPandas()

# Check Null
* there is no null value columns

In [ ]:
from pyspark.sql.functions import*
train_spark_drop.select([count(when(isnan(c), c)).alias(c) for c in train_spark_drop.columns]).toPandas()

# 3.Normalize Imbalanced data
* Most machine learning algorithms work best when the number of samples in each class are about equal. This is because most algorithms are designed to maximize accuracy and reduce error.

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

print('Histogram plot after process data set')
sns.countplot('TARGET', data=train_spark_drop.toPandas())
plt.title('Target size', fontsize=14)
plt.show()

In [ ]:
import pyspark.sql.functions as f
from pyspark.sql.window import Window
df_Target_count_2 = train_spark_drop.groupBy("TARGET").count()
df_Target_count_2.show()
df_Target_count_2 = df_Target_count_2.withColumn('ratio', f.col('count')/f.sum('count').over(Window.partitionBy()))
df_Target_count_2.orderBy('ratio', ascending=False).show()

# Resampling techniques — Undersample majority class
Stratified Sampling can be used as well 

In [ ]:
from time import *
start_time = time()

train_spark_1 = train_spark_drop.filter("TARGET =1")
train_spark_0_OG = train_spark_drop.filter("TARGET =0")
ratio = train_spark_1.count()/train_spark_0_OG.count()
print("Before Undersample 1 and 0: ",ratio)
train_spark_0, train_spark_dump = train_spark_0_OG.randomSplit([ratio,1-ratio])

#concate two dataframe together
train_spark_Undersample= train_spark_0.union(train_spark_1)
ratio_Undersample = train_spark_Undersample.filter("TARGET =1").count()/train_spark_Undersample.filter("TARGET =0").count()
print("After Undersample 1 and 0: ",ratio_Undersample)
end_time = time()
elapsed_time = end_time - start_time
print("Time for this session: %.3f seconds" % elapsed_time)

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

print('After Normalize Target distribution ')
sns.countplot('TARGET', data=train_spark_Undersample.toPandas())
plt.title('Target size', fontsize=14)
plt.show()
train_spark_Undersample.groupBy("TARGET").count().show()


# Feature Assembly

In [ ]:
feature_undersample = VectorAssembler(inputCols=train_spark_Undersample.columns[:-1],outputCol="features")
feature_vector_undersample= feature_undersample.transform(train_spark_Undersample)

# Data split

In [ ]:
(trainingData_undersample, testData_undersample) = feature_vector_undersample.randomSplit([0.8, 0.2],seed = 11)

# 4.Modelling 

# 4.1 Logistic Regression

In [ ]:
## Logistic Regression
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="TARGET", featuresCol="features", maxIter=5)
lrModel = lr.fit(trainingData_undersample)
import matplotlib.pyplot as plt
import numpy as np
beta = np.sort(lrModel.coefficients)
plt.plot(beta, label ="LogisticRegression")
plt.ylabel("Beta Coefficients")
plt.legend(loc="lower right")
plt.show()



In [ ]:
trainSet = lrModel.summary
roc = trainSet.roc.toPandas()
plt.plot(roc["FPR"],roc["TPR"], "-r", label="Logistic Regression ROC Curve")
plt.legend(loc="lower right")
plt.ylabel("False Positive Rate")
plt.xlabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()
print("TrainSet areaUnderROC: " + str(trainSet.areaUnderROC))

# 4.2 RandomForest
Random Forests are a group of decision trees, that uses Mojority of Votingfor each of the decision tree. This algorithm provides less risk of overfitting by combining decision trees.

In [ ]:
## Random Forest Classifier
from pyspark.ml.classification import RandomForestClassifier
# Creating RandomForest model.
rf = RandomForestClassifier(labelCol="TARGET", featuresCol="features", numTrees=2)
## train the model
rfModel = rf.fit(trainingData_undersample)
## make predictions
predictions = rfModel.transform(testData_undersample)
rfPredictions = predictions.select("TARGET", "prediction", "probability")
rfPredictions.show(10)


Besides Logistic Regression (like Decision Trees or Random Forest which lack a model summary), therefore I used class CurveMetrics(BinaryClassificationMetrics):

In [ ]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics
# Python: https://spark.apache.org/docs/latest/api/python/_modules/pyspark/mllib/common.html
class CurveMetrics(BinaryClassificationMetrics):
    def __init__(self, *args):
        super(CurveMetrics, self).__init__(*args)

    def _to_list(self, rdd):
        points = []
        # Note this collect could be inefficient for large datasets 
        # considering there may be one probability per datapoint (at most)
        # The Scala version takes a numBins parameter, 
        # but it doesn't seem possible to pass this from Python to Java
        for row in rdd.collect():
            # Results are returned as type scala.Tuple2, 
            # which doesn't appear to have a py4j mapping
            points += [(float(row._1()), float(row._2()))]
        return points

    def get_curve(self, method):
        rdd = getattr(self._java_model, method)().toJavaRDD()
        return self._to_list(rdd)

In [ ]:
# the probability of getting the output either as 0 or 1
# Returns as a list (false positive rate, true positive rate)
preds = predictions.select("TARGET","probability").rdd.map(lambda row: (float(row["probability"][1]), float(row["TARGET"])))
points = CurveMetrics(preds).get_curve('roc')

plt.figure()
x_val = [x[0] for x in points]
y_val = [x[1] for x in points]
plt.title("ROC")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.plot(x_val, y_val, "-r", label="Random Forest Regression ROC Curve")
plt.legend(loc="lower right")

In [ ]:
## evaluate the Rnadom Forest Classifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol="TARGET")
evaluator.evaluate(predictions)
print("Random Forest Test areaUnderROC: {}".format(evaluator.evaluate(predictions)))

# 4.3 Gradient-Boosted Tree Classifier
Gradient-Boosted Tree Classifiers are also a group of decision trees and they iteratively train decision trees in order to minimize a loss function.

In [ ]:
from pyspark.ml import Pipeline
## Gradient-Boosted Tree Classifier
from pyspark.ml.classification import GBTClassifier
stages = []
gbt = GBTClassifier(labelCol="TARGET", featuresCol="features",maxIter=5)
pipeline = Pipeline(stages=stages+[gbt])
gbtModel = pipeline.fit(trainingData_undersample)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
predictions =gbtModel.transform(testData_undersample)
# Show predictions
predictions.select("TARGET", "prediction", "probability").show(10)

In [ ]:
# the probability of getting the output either as 0 or 1
# Returns as a list (false positive rate, true positive rate)
preds_GBT = predictions.select("TARGET","probability").rdd.map(lambda row: (float(row["probability"][1]), float(row["TARGET"])))
points_GBT = CurveMetrics(preds_GBT).get_curve('roc')

plt.figure()
x_val = [x[0] for x in points_GBT]
y_val = [x[1] for x in points_GBT]
plt.title("ROC")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.plot(x_val, y_val,"-r", label="Gradient-Boosted Regression ROC Curve")
plt.legend(loc="lower right")

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="TARGET")
print("GBT Test Area Under ROC:"  + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

# 5. ParamGridBuilder and CrossValidator
 find the best model or parameters for a given dataset to improve the performance
# *I did run the code because it is time consuming and PC power.*****

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
paramGrid = ParamGridBuilder()\
 .addGrid(lr.aggregationDepth,[2,5,10])\
 .addGrid(lr.elasticNetParam,[0.0, 0.5, 1.0])\
 .addGrid(lr.fitIntercept,[False, True])\
 .addGrid(lr.maxIter,[10, 100, 1000])\
 .addGrid(lr.regParam,[0.01, 0.5, 2.0]) \
 .build()

Model tuning, it find the best model or parameters for a given dataset to improve the performance.

cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=2)
# Run cross validations
cvModel = cv.fit(trainingData_undersample)
predict_train=cvModel.transform(trainingData_undersample)
predict_test=cvModel.transform(testData_undersample)
print("Cross-validation areaUnderROC for train set is {}".format(evaluator.evaluate(predict_train)))
print("Cross-validation areaUnderROC for test set is {}".format(evaluator.evaluate(predict_test)))

# Resampling Techniques — Oversample minority class
* Oversampling can be defined as adding more copies of the minority class. Oversampling can be a good choice when you don’t have a ton of data to work with.

* panda dataframe I would use SMOTE to oversample (I used SMOTE for panda analysis which I did)

# Important Note

Always split into test and train sets BEFORE trying oversampling techniques! Oversampling before splitting the data can allow the exact same observations to be present in both the test and train sets. This can allow our model to simply memorize specific data points and cause overfitting and poor generalization to the test data.

In [ ]:
(trainingData_spark, testData_spark) = train_spark_drop.randomSplit([0.8, 0.2],seed = 11)

In [ ]:
from time import *
start_time = time()

train_spark_1_over_OG = trainingData_spark.filter("TARGET =1")
train_spark_0_over = trainingData_spark.filter("TARGET =0")
ratio = train_spark_1_over_OG.count()/train_spark_0_over.count()
print("Before oversample ratio 1 and 0: ",ratio)
sampleRatio = train_spark_0_over.count()/trainingData_spark.count()
print("sampleRatio:", sampleRatio)

# duplicate the minority rows
# explode_range
explode_range = range(int(train_spark_0_over.count()/train_spark_1_over_OG.count()))
train_spark_1_over = train_spark_1_over_OG.withColumn("dummy", explode(array([lit(x) for x in explode_range]))).drop('dummy')
#print(train_spark_1_over.count())

#concate two dataframe together
train_spark_oversample= train_spark_1_over.union(train_spark_0_over)
ratio_oversample = (train_spark_oversample.filter("TARGET =1")).count()/(train_spark_oversample.filter("TARGET =0")).count()
print("After oversample ratio 1 and 0: ",ratio_oversample)
end_time = time()
elapsed_time = end_time - start_time
print("Time for this session: %.3f seconds" % elapsed_time)

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

print('After Normalize Target distribution ')
sns.countplot('TARGET', data=train_spark_oversample.toPandas())
plt.title('Target size', fontsize=14)
plt.show()
train_spark_oversample.groupBy("TARGET").count().show()

# Feature assembly

In [ ]:
# train_spark_oversample, testData_spark
feature_oversample_train = VectorAssembler(inputCols=train_spark_oversample.columns[:-1],outputCol="features")
trainingData_oversample= feature_oversample_train.transform(train_spark_oversample)

feature_oversample_test = VectorAssembler(inputCols=testData_spark.columns[:-1],outputCol="features")
testData_oversample= feature_oversample_test.transform(testData_spark)


#trainingData_oversample, testData_oversample





# 6. Modelling
# 6.1 Logistic Regression

In [ ]:
## Logistic Regression
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="TARGET", featuresCol="features", maxIter=5)
lrModel = lr.fit(trainingData_oversample)
import matplotlib.pyplot as plt
import numpy as np
beta = np.sort(lrModel.coefficients)
plt.plot(beta, label ="LogisticRegression")
plt.ylabel("Beta Coefficients")
plt.legend(loc="lower right")
plt.show()

In [ ]:
trainSet = lrModel.summary
roc = trainSet.roc.toPandas()
plt.plot(roc["FPR"],roc["TPR"], "-r", label="Logistic Regression ROC Curve")
plt.legend(loc="lower right")
plt.ylabel("False Positive Rate")
plt.xlabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()
print("TrainSet areaUnderROC: " + str(trainSet.areaUnderROC))

# 6.2 RandomForest

In [ ]:
#trainingData_oversample, testData_oversample
## Random Forest Classifier
from pyspark.ml.classification import RandomForestClassifier
# Creating RandomForest model.
rf = RandomForestClassifier(labelCol="TARGET", featuresCol="features", numTrees=2)
## train the model
rfModel = rf.fit(trainingData_oversample)
## make predictions
predictions = rfModel.transform(testData_oversample)
rfPredictions = predictions.select("TARGET", "prediction", "probability")
rfPredictions.show(10)

In [ ]:
# the probability of getting the output either as 0 or 1
# Returns as a list (false positive rate, true positive rate)
preds = predictions.select("TARGET","probability").rdd.map(lambda row: (float(row["probability"][1]), float(row["TARGET"])))
points = CurveMetrics(preds).get_curve('roc')

plt.figure()
x_val = [x[0] for x in points]
y_val = [x[1] for x in points]
plt.title("ROC")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.plot(x_val, y_val, "-r", label="Random Forest Regression ROC Curve")
plt.legend(loc="lower right")

In [ ]:
## evaluate the Rnadom Forest Classifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol="TARGET")
evaluator.evaluate(predictions)
print("Random Forest Test areaUnderROC: {}".format(evaluator.evaluate(predictions)))

# 6.3 Gradient-Boosted Tree Classifier

In [ ]:
#trainingData_oversample, testData_oversample
from pyspark.ml import Pipeline
## Gradient-Boosted Tree Classifier
from pyspark.ml.classification import GBTClassifier
stages = []
gbt = GBTClassifier(labelCol="TARGET", featuresCol="features",maxIter=5)
pipeline = Pipeline(stages=stages+[gbt])
gbtModel = pipeline.fit(trainingData_oversample)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
predictions =gbtModel.transform(testData_oversample)
# Show predictions
predictions.select("TARGET", "prediction", "probability").show(10)

In [ ]:
# the probability of getting the output either as 0 or 1
# Returns as a list (false positive rate, true positive rate)
preds_GBT = predictions.select("TARGET","probability").rdd.map(lambda row: (float(row["probability"][1]), float(row["TARGET"])))
points_GBT = CurveMetrics(preds_GBT).get_curve('roc')

plt.figure()
x_val = [x[0] for x in points_GBT]
y_val = [x[1] for x in points_GBT]
plt.title("ROC")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.plot(x_val, y_val,"-r", label="Gradient-Boosted Regression ROC Curve")
plt.legend(loc="lower right")

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="TARGET")
print("GBT Test Area Under ROC:"  + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

warning-----------------------------------------------------------------------------------------------warning
*  code below is just extra things which I couldn't utilize. Could have good usage

# Feature selection Matrix (extra need bit more understanding and usage)
* find it interesting, but no time to understand optimiza the usage
* try it out with undersample data, since the data volumn is not high.

In [ ]:
# inspire from https://stackoverflow.com/questions/51831874/how-to-get-correlation-matrix-values-pyspark/51834729
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

#drop target column
corr_df = train_spark_oversample.drop("TARGET")
# copying columns names
column_names = corr_df.columns


# get correlation matrix
#matrix = Correlation.corr(feature_vector)


vector_col = "corr_features"
assembler = VectorAssembler(inputCols=train_spark_oversample.columns[:-1], 
                            outputCol=vector_col)
feature_vector = assembler.transform(train_spark_oversample).select(vector_col)
matrix = Correlation.corr(feature_vector, vector_col)

# Setting column names of datafram

convert_matrix = (matrix.collect()[0][0]).toArray().tolist()
df_matrix  = pd.DataFrame( convert_matrix, column_names)





In [ ]:
df_matrix  = pd.DataFrame( convert_matrix, column_names,column_names)

In [ ]:
df_matrix

In [ ]:
import matplotlib.ticker as ticker
import matplotlib.cm as cm
import matplotlib as mpl

import matplotlib.pyplot as plt
%matplotlib inline


df_matrix_04 = df_matrix[(df_matrix[:]>0.5)|(df_matrix[:]<-0.5)]
df_matrix_04.head(20)
fig, ax = plt.subplots(figsize=(10,10)) 
sns.heatmap(df_matrix_04, cmap="YlGnBu")



In [ ]:
# collection of coloumns which need to be reomoved 

cols_to_drop_1 = []
 
for col in range(len(df_matrix.columns)):
    for row in range(col):
        if (df_matrix.iloc[row,col] >0.5 \
            or df_matrix.iloc[row,col] < -0.5) \
            and (df_matrix.columns[row] not in cols_to_drop_1):
            cols_to_drop_1.append(df_matrix.columns[col])

train_spark_filter = train_spark_oversample.drop(*cols_to_drop_1)

print("Number of cols dropped: ",  len(cols_to_drop_1))
print("Number of cols train_spark_filter: ",  len(train_spark_filter.columns))
# there is duplicate coloumn name therefore the size the big, need sometime to look into this. 
